VERSION 1.0 ON 28072022 - 22.52

VERSION 1.0.01 ON 29072022 - 00.14 - adjusted output .pdf image size

VERSION 1.0.2  ON 29072022 - 19.27 - plotted leftover graphs

VERSION 1.0.21 ON 30072022 - 19.33 - corrected page order in the final output file

VERSION 1.0.22 ON 31072022 - 16.50 - changed log10(fulx_err) tolerance limit to -11 and minor changes to file titles

I HAVE BEEN ALLOTTED 80 GRB FILES (NAMES STORED IN THE VARIABLE $folders$)

OUT OF THOSE 80 FILES, 22 ARE MASTERSAMPLE FILES SO THEY HAVE BEEN IGNORED AS PER INSTRUCTIONS

THE REMAINING 52 FILES ARE WITHOUT DUPLICATE BANDS IN THE SAME TIME AND/OR LIMITING MAGNITUDES

ANY POINTS SEEN OVERLAPPING ARE SIMPLY VERY CLOSE TO EACH OTHER

In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
import kaleido
from PyPDF2 import PdfFileMerger

In [3]:
#FILE PATHS
systempath = "/media/darkwake/VIB2/"
inputfilepath = systempath + "GRB/ALL_LCS_FOR_500/"
output_file_path = systempath + "task2/v_converted_filtered_files/"
output_pdf_path = systempath + "GRB/task2/v_out_pdfs/"

#THE 80 FILES ALOTTED TO ME
folders = "140423A ,140428A ,140430A ,140506A ,140508A ,140509A ,140512A ,140515A ,140518A ,140606B ,140614A ,140620A ,140623A ,140629A ,140703A ,140710A ,140801A ,140808A ,140903A ,140907A ,141004A ,141028A ,141109A ,141121A ,141220A ,141221A ,141225A ,150101B ,150120B ,150206A ,150301B ,150314A ,150323A ,150403A ,150413A ,150423A ,150514A ,150518A ,150727A ,150818A ,150821A ,150910A ,150915A ,151021A ,151027A ,151027B ,151029A ,151031A ,151111A ,151112A ,151215A ,160117B ,160121A ,160131A ,160203A ,160227A ,160228A ,160314A ,160410A ,160425A ,160509A ,160623A ,160625B ,160629A ,160804A ,160821B ,161014A ,161017A ,161023A ,161108A ,161117A ,161129A ,161219B ,170113A ,170202A ,170214A ,170405A ,170428A ,170519A"

#22 MASTERSAMPLE FILES WITHIN MY ALOTTED FILES
masterfolders = "140423A,140430A,140801A,140907A,141121A,141221A,150323A,150413A,150910A,151027A,151029A,160131A,160227A,160314A,160425A,160804A,161014A,161023A,161219B,170113A,170405A, 170531B"
folders = folders.split(" ,")
masterfolders = masterfolders.split(",")


#ADDITIONAL PRECURSOR FILES
v_folders = ['121027A', '121201A', '121209A', '121211A', '121229A', '130408A', '130418A', '130427B', '130505A', '130511A', '130514A', '130518A', '130603B', '130610A', '130612A', '130701A', '130907A', '130925A', '131004A', '131011A', '131030A', '131103A', '131105A', '131108A', '131117A', '131231A', '140114A', '140213A', '140226A', '140301A', '140304A', '140311A', '140318A', '140419A']

#Biagio's Additional files - are merged in a separate .pdf
b_folders = ['970228A', '971214A', '980329A', '980425A', '980613A', '980703A', '990123A', '990712A', '991208A']
#REMOVING THE MASTERSAMPLE FILES FROM THE LIST OF FILES TO BE SCANNED
for i in masterfolders:
    if i in folders:
        folders.remove(i)
v_folders.extend(folders)
filenames = [i + "_converted_flux" for i in v_folders]
b_filenames = [i + "_converted_flux" for i in b_folders]

FUNCTION TO REMOVE POINTS WITH 0 ERROR(LIMITING VALUE ERROR) AND CORRUPTED DATA AND (FROM VERSION 1.0.22) POINTS WITH TOO MUCH FLUX_ERROR

In [4]:
def zero_error(light):
    for i in range(len(light)):
        if light.flux_err.values[i] == 0.0 or np.log10(light.flux_err.values[i]) > -11:#Version 1.0.22 changed log10(fulx_err) tolerance limit to -11 
            light.flux_err.values[i] = None
        if type(light.index.values[i]) == str:
            light.flux_err.values[i] = None
    light.dropna(inplace=True)       

IF AN OBSERVATION HAS BEEN DONE IN MULTIPLE BANDS AT THE SAME TIME,

THIS METHOD REMOVES ALL THE BANDS EXCEPT R / r / r' BANDS FROM THAT OBSERVATION

In [5]:
def vertical(light):
    disc = dict(light.index.value_counts())
    disc_time = list(disc.keys())
    val_counts = list(disc.values())
    for i in range(len(disc_time)):
        if val_counts[i] > 1:
            for j in range(len(light)):
                if light.index.values[j] == disc_time[i]:
                    if not(light.band.values[j] in ("R","r", "r'", "R'")):
                        light.band.values[j] = None
                        #light.index.values[j] = None

    light.dropna(inplace=True)

THE FOLLOWING METHOD CLEANS A GIVEN INPUT FILE AND STORES THE CLEANED DATA FILE IN $task2/v$ _ $converted$ _ $filtered$ _ $files$ FOLDER

In [6]:
def filter(f2):
     filename = inputfilepath + f2
     filename2 = f2.replace( "_flux","_filtered_flux")
     try:
          light = pd.read_csv(filename+".txt", sep=r"\t+|\s+", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)
          zero_error(light)
          vertical(light)
        
          light.to_csv("task2/v_converted_filtered_files/"+filename2 + ".txt")
          return light

     except:
          light = pd.read_csv(filename+".txt", sep=r"\t+|\s+", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)
          zero_error(light)
          vertical(light)
        
          light.to_csv("task2/v_converted_filtered_files/"+filename2 + ".txt")
          return light


FUNCTION TO PLOT A GIVEN DATABASE AND STORE THE OUTPUT PLOT IN $task2/v$ _ $out$ _ $pdfs$ FOLDER

In [7]:
def plot(light, f2):
    #filename2 = file + "_not_done"
    filename2 = f2.replace( "_flux","_filtered_flux")
    filename = f2
    if len(light) != 0:
        try:
            fig = px.scatter(
                        data_frame=light,
                        x=np.log10(light.index.values),
                        y=np.log10(light[1].values),
                        error_y= light[2].values / (light[1].values * np.log(10)),
                        color=light[3].values,
                        hover_data=[4],
                        title=filename2
                    )

            fig.update_layout(
                        xaxis_title='log Time (s)',
                        yaxis_title='log Flux (erg cm-2 s-1)'
                    )
            fig.write_image(output_pdf_path + filename2+".pdf")

            
        except:
            fig = px.scatter(
                data_frame=light,
                x=np.log10(light.index),
                y=np.log10(light['flux'].values),
                error_y= light['flux_err'].values / (light['flux'].values * np.log(10)),
                color=light['band'].values,
                hover_data=['source'],
                title=filename2, width=1200, height=800
            )
            fig.updatetask3/v_converted_filtered_files

In [8]:
file = filenames[2]
plot(filter(file), file)

AttributeError: 'Figure' object has no attribute 'updatetask3'

In [17]:
filter(file)

,flux,flux_err,band,source
time_sec,,,,
9720.0,1.440026e-14,2.727879e-17,r,14048
9540.0,4.348044e-15,1.201437e-15,r',14049
4896.0,4.366407e-15,8.079542e-16,I,14056
13392.0,3.298329e-14,6.205965e-17,r',14050
20803.0,1.365271e-13,1.850492e-15,v_swift,14086
15083.5,8.889499e-14,2.763591e-15,b_swift,14086
15502.5,2.481220e-14,1.189791e-15,u_swift,14086
18101.5,2.261144e-14,1.574753e-15,uvw1_swift,14086
21265.0,9.235545e-15,7.748315e-16,uvm2_swift,14086


In [1]:
plot(filter(file), file)

NameError: name 'plot' is not defined